# Regresión losgística para análisis de sentimientos

In [1]:
import sys
import os
ROOT_PATH = os.path.join(sys.path[0].split('BecaNLP')[0],'BecaNLP')
DATASETS_ROOT_PATH = os.path.join(ROOT_PATH,'Utils/Datasets')
NLPUTILS_PATH = os.path.join(ROOT_PATH,'Utils/')
sys.path += [DATASETS_ROOT_PATH, NLPUTILS_PATH]

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import torch
%matplotlib notebook
import matplotlib.pyplot as plt

from NLPUtils import *
                
%load_ext autoreload
%autoreload 2

En este notebook se va a estudiar el modelo de Regresión Logística aplicado a análisis de sentimientos.

## Problema general

Vamos a formular el problema de análisis de sentimientos como un problema de clasificación binaria. Se dispondrá de un conjunto de datos de entrenamiento que consisten en una serie de comentarios conformados por una secuencia de palabras, y su respectiva connotación (positiva o negativa). El objetivo de esta parte será estudiar el modelo de **regresión logística** utilizado para resolver este problema.

Antes de continuar, detallamos la notación que vamos a utilizar:

* El texto de entrada va a ser mapeado a un ejemplo de un vector aleatorio $\mathbf{x}=\begin{bmatrix} x_1 & x_2 & \ldots & x_n \end{bmatrix}^T$ que representa el conjunto de features de ese texto.

* La clase a la que pertenece cada comentario se denota con un ejemplo de una variable aleatoria $y$, con realizaciones en el conjunto $\{ 0,1 \}$. De esta manera, hacemos corresponder a $y=1$ cuando el texto tiene una connotación positiva y a $y=0$ cuando ésta es negativa.

* El conjunto de $N$ muestras de entrenamiento se representa por $\left\{ \left(\mathbf{x}^{(i)}, y^{(i)}\right) \right\}_{i=1}^N$.

## El modelo

El modelo de regresión logística es un modelo discriminativo que forma parte de la familia de modelos discriminativos paramétricos conocida como **Modelos Lineales Generalizados** (*Generalized Linear Models*, GLM). Un modelo de este tipo puede construirse de la siguiente manera:

1. Se define que la variable aleatoria $y|\mathbf{x}$ que relaciona la entrada con la salida depende de un conjunto de parámetros $\theta$ y pertenece a la familia de exponenciales de parámetro $\eta$ de tal manera que $\eta = \theta^T \mathbf{x}$. Esto es:

$$
\begin{align*}
y|\mathbf{x} &\sim \mathrm{ExpFamily}\left(\eta\right) \\[.5em]
P(y|\mathbf{x}) &= b(y)\exp\left( \eta^T y - a\left(\eta\right)\right) \\[.5em]
P(y|\mathbf{x};\theta) &= b(y)\exp\left( \mathbf{x}^T \theta y - a(\theta^T \mathbf{x})\right)
\end{align*}
$$

2. La salida del modelo se obtiene por medio de

$$
h_\theta(\mathbf{x}) = E\left[ y|\mathbf{x}\right]
$$

El modelo de regresión logistica consiste en definir la probabilidad a posteriori 

$$
P(y|\mathbf{x};\theta) = \sigma\left( \theta^T \mathbf{x} \right)^y \left( 1 - \sigma\left( \theta^T \mathbf{x} \right) \right)^{(1-y)}
$$

donde 

$$
\sigma(z) = \frac{1}{1+ e^{-z}}
$$ 

y puede mostrarse que forma parte de los modelos GLM para el caso en que $T(y)=y$, $b(y)=1$ y $a(\theta^T\mathbf{x})=\log\left(1+e^{\theta^T\mathbf{x}}\right)$. 

También puede verse que 

$$
h_\theta(\mathbf{x}) = E\left[ y|\mathbf{x}\right] = P(y=1|\mathbf{x};\theta)\cdot 1 + P(y=0|\mathbf{x};\theta)\cdot 0 = \sigma\left( \theta^T \mathbf{x} \right)
$$

por lo que la salida del modelo da la probabilidad de pertenecer a la clase $y=1$, y por lo tanto, aporta todo lo necesario para realizar una nueva predicción.

En lo que sigue, adoptaremos el criterio de **máxima verosimilitud** para estimar los parámetros del modelo. De esta forma:

$$
\begin{align*}
\hat{\theta} &= \mathrm{argmax}_\theta \prod_{i=1}^N P\left(y^{(i)}|\mathbf{x}^{(i)};\theta\right) \\
&= \mathrm{argmax}_\theta \prod_{i=1}^N \sigma\left( \theta^T \mathbf{x}^{(i)} \right)^{y^{(i)}} \left( 1 - \sigma\left( \theta^T \mathbf{x}^{(i)} \right) \right)^{(1-y^{(i)})}\\
&= \mathrm{argmax}_\theta \log\left(\prod_{i=1}^N \sigma\left( \theta^T \mathbf{x}^{(i)} \right)^{y^{(i)}} \left( 1 - \sigma\left( \theta^T \mathbf{x}^{(i)} \right) \right)^{(1-y^{(i)})}\right)\\
&= \mathrm{argmax}_\theta \sum_{i=1}^N y^{(i)} \log\left(\sigma\left( \theta^T \mathbf{x}^{(i)} \right)\right) + \left(1 - y^{(i)}\right) \log\left(1 - \sigma\left( \theta^T \mathbf{x}^{(i)} \right)\right)\\
\end{align*}
$$

## Extracción de features

Si bien extraer características es el principal problema en procesamiento de lenguaje, en este caso sólo se experimentará con una serie de variantes en la tarea de representar un texto como una serie de características o *features*. Sin embargo, la parte en la que se comprende lo que se está diciendo en el texto está incluída, sin duda, en la extracción de features del texto. 

### Bolsa de Palabras

Una forma muy común de representar un texto es con una **Bolsa de Palabras** (*Bag of Words*, BOW). Este método consiste en definir un vocabulario de palabras $V=\{ w_1, w_2,\ldots, w_n\}$ y contar la cantidad de veces que apareció cada una de estas palabras en el texto. De esta manera, la $i$-ésima coordenada del vector $\mathbf{x}$ corresponde a la cantidad de veces que apareció la palabra $w_i$ en el texto.

Por ejemplo, supongamos que se tiene el siguiente texto:

```
<START> I am Sam. Sam I am. I do not like green eggs and ham. <END>
```

y se define un vocabulario `V = ['I', 'am', 'Sam', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '.', '<START>', '<END>']`. Entonces, el vector de features que representa el texto anterior es

$$
x = \begin{bmatrix}
count(I) \\
count(am) \\
count(Sam) \\
\vdots \\
count(.) \\
count(<START>) \\
count(<END>) 
\end{bmatrix}
=
\begin{bmatrix}
3 \\ 2 \\ 2 \\ \vdots \\ 3 \\ 1 \\ 1 
\end{bmatrix}
$$

Notemos varias cosas:

* Esta representación es equivalente a representar un histograma de las palabras de que aparecen en cada muestra.

* No se tiene en cuenta el orden en el que aparecen las palabras, por lo que se está perdiendo información (¡y muy valiosa!)

* Existen elementos del vocabulario, como el punto y los signos de comienzo y fin del texto, que no son palabras pero que, sin embargo, forma parte del texto. Esto se hace porque aportan información valiosa sobre el texto, y descartarlos, muchas veces disminuye el desempeño del algoritmo. Por otro lado, se verá que tener muchos componentes en el vocabulario también puede jugar en contra de la cantidad de aciertos. Es común denominar a los integrantes del vocabulario ***tokens*** cuando se refiere a los ejemplos de estos elementos en el texto, y ***types*** cuando se refieren a los elementos en sí.

* Existe la posibilidad de que en el texto aparezcan tokens (como por ejemplo `green`) que no forman parte del vocabulario. En este caso se suelen ignorar estas apariciones, aunque a veces se suele incorporar un token especial de tipo desconocido (representado como `<UNK>`) que contabilice las palabras que están fuera del vocabulario. 